COCO JSON → YOLO 학습용 형식 변환

1. images/train/*.png

2. labels/train/*.txt ← YOLO 라벨 (class x_center y_center width height)

3. dataset.yaml ← YOLO 학습 설정 파일

yolo format: Label ID, Center x, Center y, Width, Height

YOLO는 이미지 크기와 관계없이 학습할 수 있도록, 모든 값이 상대 좌표로 들어감
중심 좌표 기준이기 때문에 변환이 필요함

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jihyungpang","key":"2d5e377d246a0d9d8c94fe118214f71b"}'}

In [2]:
import os
import zipfile

# kaggle.json을 숨은 디렉토리에 복사
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# 권한 설정
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install -q kaggle

In [4]:
!kaggle competitions download -c ai02-level1-project

100% 3.90G/3.91G [00:33<00:00, 155MB/s]
100% 3.91G/3.91G [00:33<00:00, 126MB/s]


In [5]:
!unzip -q ai02-level1-project.zip -d ./ai02_data

In [6]:
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 72.0 MB/s eta 0:00:00


In [7]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
from tqdm import tqdm
import cv2

In [8]:
train_image_dir = './ai02_data/train_images'
test_image_dir = './ai02_data/test_images'
annotation_dir = './ai02_data/train_annotations'

In [9]:
# Load all annotation JSONs
all_images = []
all_annotations = []
all_categories = []

for root, _, files in os.walk(annotation_dir):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for img in data.get("images", []):
                    img["source_file"] = file
                all_images.extend(data.get("images", []))
                all_annotations.extend(data.get("annotations", []))
                all_categories.extend(data.get("categories", []))

df_images = pd.DataFrame(all_images)
df_annotations = pd.DataFrame(all_annotations)
df_categories = pd.DataFrame(all_categories).drop_duplicates("id")

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
df_images.head()

file_name  width  height  \
0  K-003351-016232-019232_0_2_0_2_90_000_200.png    976    1280   
1  K-003351-016232-019232_0_2_0_2_70_000_200.png    976    1280   
2  K-003351-016232-019232_0_2_0_2_75_000_200.png    976    1280   
3  K-003351-016232-019232_0_2_0_2_90_000_200.png    976    1280   
4  K-003351-016232-019232_0_2_0_2_70_000_200.png    976    1280   

                                         imgfile    drug_N drug_S back_color  \
0  K-003351-016232-019232_0_2_0_2_90_000_200.png  K-019232   정상알약     연회색 배경   
1  K-003351-016232-019232_0_2_0_2_70_000_200.png  K-019232   정상알약     연회색 배경   
2  K-003351-016232-019232_0_2_0_2_75_000_200.png  K-019232   정상알약     연회색 배경   
3  K-003351-016232-019232_0_2_0_2_90_000_200.png  K-003351   정상알약     연회색 배경   
4  K-003351-016232-019232_0_2_0_2_70_000_200.png  K-003351   정상알약     연회색 배경   

  drug_dir light_color  camera_la  camera_lo  size dl_idx dl_mapping_code  \
0       앞면         주백색         90          0   200  19231        K-019232   
1       앞면         주백색         70          0   200  19231        K-019232   
2       앞면         주백색         75          0   200  19231        K-019232   
3       앞면         주백색         90          0   200   3350        K-003351   
4       앞면         주백색         70          0   200   3350        K-003351   

           dl_name              dl_name_en  \
0  콜리네이트연질캡슐 400mg     Cholinate Soft Cap.   
1  콜리네이트연질캡슐 400mg     Cholinate Soft Cap.   
2  콜리네이트연질캡슐 400mg     Cholinate Soft Cap.   
3      일양하이트린정 2mg  Hytrin Tab. 2mg Ilyang   
4      일양하이트린정 2mg  Hytrin Tab. 2mg Ilyang   

                                             img_key dl_material  \
0  http://connectdi.com/design/img/drug/1NOwp2F6D...    콜린알포세레이트   
1  http://connectdi.com/design/img/drug/1NOwp2F6D...    콜린알포세레이트   
2  http://connectdi.com/design/img/drug/1NOwp2F6D...    콜린알포세레이트   
3  http://connectdi.com/design/img/drug/147765842...  테라조신염산염수화물   
4  http://connectdi.com/design/img/drug/147765842...  테라조신염산염수화물   

                    dl_material_en dl_custom_shape dl_company  \
0              Choline Alfoscerate           연질캡슐제    한미약품(주)   
1              Choline Alfoscerate           연질캡슐제    한미약품(주)   
2              Choline Alfoscerate           연질캡슐제    한미약품(주)   
3  Terazosin Hydrochloride Hydrate         정제, 저작정    일양약품(주)   
4  Terazosin Hydrochloride Hydrate         정제, 저작정    일양약품(주)   

           dl_company_en di_company_mf di_company_mf_en   item_seq  \
0            Hanmi Pharm                                 200605445   
1            Hanmi Pharm                                 200605445   
2            Hanmi Pharm                                 200605445   
3  Ilyang Pharmaceutical                                 199101125   
4  Ilyang Pharmaceutical                                 199101125   

  di_item_permit_date               di_class_no di_etc_otc_code  \
0            20060616         [01190]기타의 중추신경용약           전문의약품   
1            20060616         [01190]기타의 중추신경용약           전문의약품   
2            20060616         [01190]기타의 중추신경용약           전문의약품   
3            19910626  [02590]기타의 비뇨생식기관 및 항문용약           전문의약품   
4            19910626  [02590]기타의 비뇨생식기관 및 항문용약           전문의약품   

           di_edi_code                            chart drug_shape  thick  \
0  643502600,A21404481  무색투명의 액을 넣은 연한 노란색을 띤 타원형의 연질캡슐        타원형   9.00   
1  643502600,A21404481  무색투명의 액을 넣은 연한 노란색을 띤 타원형의 연질캡슐        타원형   9.00   
2  643502600,A21404481  무색투명의 액을 넣은 연한 노란색을 띤 타원형의 연질캡슐        타원형   9.00   
3  641702010,A09503981               오랜지색 또는 노란색의 원형정제.         원형   2.76   
4  641702010,A09503981               오랜지색 또는 노란색의 원형정제.         원형   2.76   

   leng_long  leng_short print_front print_back color_class1 color_class2  \
0      14.30        9.00       HM-CN                      갈색                
1      14.30        9.00       HM-CN                      갈색                
2      14.30        9.00       HM-CN                      갈색                
3       7.01        

In [12]:
df_annotations.head()

area  iscrowd                  bbox  category_id  ignore segmentation  \
0  87579        0  [146, 806, 333, 263]        19231       0           []   
1  83640        0  [497, 235, 328, 255]        19231       0           []   
2  89496        0  [103, 704, 339, 264]        19231       0           []   
3  32942        0  [379, 217, 182, 181]         3350       0           []   
4  36288        0  [413, 883, 189, 192]         3350       0           []   

     id  image_id  
0  5431      1428  
1  5429      1426  
2  5430      1427  
3  5428      1428  
4  5426      1426

In [13]:
df_annotations.shape

(4526, 8)

In [14]:
df_categories.head()

supercategory     id             name
0           pill  19231  콜리네이트연질캡슐 400mg
3           pill   3350      일양하이트린정 2mg
6           pill  16231        리피토정 20mg
9           pill   5093    삼남건조수산화알루미늄겔정
11          pill   2482      뮤테란캡슐 100mg

In [15]:
unique_categories = sorted(df_categories["id"].unique())
category_id_to_class_id = {cat_id: idx for idx, cat_id in enumerate(unique_categories)}

In [16]:
# 새로
label_dir = "./labels"
os.makedirs(label_dir, exist_ok=True)

for image_id in df_images["id"].unique():
    image_info = df_images[df_images["id"] == image_id].iloc[0]
    annotations = df_annotations[df_annotations["image_id"] == image_id]

    label_lines = []
    for _, ann in annotations.iterrows():
        x, y, w, h = ann["bbox"]
        img_w, img_h = image_info["width"], image_info["height"]

        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        class_id = category_id_to_class_id[ann["category_id"]]
        line = f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
        label_lines.append(line)

    image_filename = os.path.splitext(image_info["file_name"])[0]
    label_path = os.path.join(label_dir, image_filename + ".txt")

    with open(label_path, "w") as f:
        f.write("\n".join(label_lines))

In [ ]:
import shutil
from google.colab import files

# 라벨 디렉토리를 zip으로 압축
shutil.make_archive("labels_yolo", "zip", label_dir)

# 압축된 zip 파일 다운로드
files.download("labels_yolo.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
import os
import shutil
import random
from glob import glob

# 이미지 디렉토리 및 라벨 디렉토리 경로
image_dir = train_image_dir  # 예: "/content/ai02_data/train_images"
label_dir = "./labels"       # 예: "/content/ai02_data/labels/train"
#test_image_dir
# 결과 저장 디렉토리 설정
output_base = "./yolo_dataset"
img_train_dir = os.path.join(output_base, "images", "train")
img_val_dir = os.path.join(output_base, "images", "val")

lbl_train_dir = os.path.join(output_base, "labels", "train")
lbl_val_dir = os.path.join(output_base, "labels", "val")

# 디렉토리 생성
for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    os.makedirs(d, exist_ok=True)

# 이미지 파일 경로 리스트 만들기
image_paths = sorted(glob(os.path.join(image_dir, '*.png')))
random.seed(71)  # 재현 가능성 확보
random.shuffle(image_paths)

# 8:2로 나누기
val_count = int(len(image_paths) * 0.2)
val_paths = image_paths[:val_count]
train_paths = image_paths[val_count:]

# 복사 함수 정의
def copy_files(path_list, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    for img_path in path_list:
        file_name = os.path.basename(img_path)
        base_name = os.path.splitext(file_name)[0]

        # 이미지 복사
        shutil.copy2(img_path, os.path.join(dst_img_dir, file_name))

        # 라벨 경로
        src_lbl_path = os.path.join(src_lbl_dir, base_name + '.txt')
        dst_lbl_path = os.path.join(dst_lbl_dir, base_name + '.txt')

        # 라벨 파일이 존재할 경우에만 복사
        if os.path.exists(src_lbl_path):
            shutil.copy2(src_lbl_path, dst_lbl_path)

# 복사 실행
copy_files(train_paths, label_dir, img_train_dir, lbl_train_dir)
copy_files(val_paths, label_dir, img_val_dir, lbl_val_dir)

print(f"Train 이미지 수: {len(train_paths)}")
print(f"Val 이미지 수: {len(val_paths)}")

Train 이미지 수: 1192
Val 이미지 수: 297


In [18]:
import os

output_base = "./yolo_dataset"
yaml_path = os.path.join(output_base, "dataset.yaml")

# df_categories에서 id 순서대로 정렬 후 클래스명 리스트 생성
df_categories_sorted = df_categories.sort_values("id")
class_names = df_categories_sorted["name"].tolist()
num_classes = len(class_names)

yaml_content = f"""
train: {os.path.join(output_base, "images/train")}
val: {os.path.join(output_base, "images/val")}

nc: {num_classes}
names: {class_names}
"""

# output_base 폴더가 없으면 생성
os.makedirs(output_base, exist_ok=True)

with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())

print(f"YAML 파일이 생성되었습니다: {yaml_path}")

YAML 파일이 생성되었습니다: ./yolo_dataset/dataset.yaml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
# output_base 폴더를 yolo_dataset.zip으로 압축
shutil.make_archive("yolo_dataset", 'zip', output_base)

files.download("yolo_dataset.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [20]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [21]:
model = YOLO("yolov8n.pt")
model.info()

100%|██████████| 6.25M/6.25M [00:00<00:00, 134MB/s]


YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [22]:
import torch
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("사용 중인 디바이스:", torch.cuda.get_device_name(0))

CUDA 사용 가능 여부: True
사용 중인 디바이스: NVIDIA L4


In [23]:
results = model.train(
    data='/content/yolo_dataset/dataset.yaml',  # 너가 만든 YAML 경로로 수정
    epochs=100,
    batch=16,  # GPU 메모리에 따라 조정
    imgsz=640,
    device=0,
    cache=True
)

Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

100%|██████████| 22.2M/22.2M [00:00<00:00, 112MB/s] 

Overriding model.yaml nc=80 with nc=73

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,096,741 parameters, 3,096,725 gradients, 8.6 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 108MB/s]


...

    100/100      2.74G     0.1865     0.6136     0.7926         19        640: 100%|██████████| 75/75 [00:07<00:00, 10.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  6.95it/s]

                   all        297        910      0.785      0.966      0.868      0.862



100 epochs completed in 0.258 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,091,487 parameters, 0 gradients, 8.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.37it/s]


                   all        297        910      0.783       0.98      0.875      0.868
            보령부스파정 5mg         41         41      0.831          1      0.873      0.869
           뮤테란캡슐 100mg         40         40      0.813      0.977      0.877      0.866
           일양하이트린정 2mg         44         44      0.809          1       0.86      0.854
    기넥신에프정(은행엽엑스)(수출용)         95         95      0.823          1       0.83      0.823
     무코스타정(레바미피드)(비매품)         23         23      0.766          1      0.819      0.819
                  알드린정         10         10      0.575          1      0.812      0.812
          뉴로메드정(옥시라세탐)          2          2      0.788          1      0.995      0.945
            타이레놀정500mg          8          8      0.864          1      0.939      0.908
          에어탈정(아세클로페낙)          9          9      0.675          1       0.83       0.83
         삼남건조수산화알루미늄겔정          9          9      0.797          1      0.875      0.875
타이레놀이알서방정(아세트아미노펜)(수출

In [24]:
train_results = model.predict(
    source="/content/yolo_dataset/images/train",     # 테스트 이미지 디렉토리
    conf=0.25,                  # confidence threshold
    imgsz=640,                  # 이미지 크기
    save=True,                  # 예측 결과 이미지 저장
    save_txt=True,              # 바운딩 박스 txt 저장
    show_labels=True,           # 이미지에 라벨 표시
    show_conf=True,             # confidence score 표시
    name="predict"              # 저장 디렉토리 이름 (runs/detect/predict)
)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1192 /content/yolo_dataset/images/train/K-001900-010224-016551-031705_0_2_0_2_70_000_200.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 낙소졸정 500/20mg, 65.6ms
image 2/1192 /content/yolo_dataset/images/train/K-001900-010224-016551-031705_0_2_0_2_90_000_200.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 낙소졸정 500/20mg, 8.3ms
image 3/1192 /content/yolo_dataset/images/train/K-001900-010224-016551-033009_0_2_0_2_70_000_200.png: 640x512 1 보령부스파정 5mg, 1 동

In [26]:
model = YOLO("runs/detect/train/weights/best.pt")
metrics = model.val()
print(metrics.box)

Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,091,487 parameters, 0 gradients, 8.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4602.1±779.2 MB/s, size: 1731.4 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 297 images, 0 backgrounds, 0 corrupt: 100%|██████████| 297/297 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03<00:00,  4.97it/s]


                   all        297        910      0.783       0.98      0.875      0.868
            보령부스파정 5mg         41         41      0.831          1      0.874       0.87
           뮤테란캡슐 100mg         40         40      0.813      0.978      0.875      0.864
           일양하이트린정 2mg         44         44      0.809          1       0.86      0.854
    기넥신에프정(은행엽엑스)(수출용)         95         95      0.823          1       0.83      0.823
     무코스타정(레바미피드)(비매품)         23         23      0.766          1      0.819      0.819
                  알드린정         10         10      0.575          1      0.808      0.808
          뉴로메드정(옥시라세탐)          2          2      0.788          1      0.995      0.945
            타이레놀정500mg          8          8      0.864          1      0.939      0.908
          에어탈정(아세클로페낙)          9          9      0.675          1       0.83       0.83
         삼남건조수산화알루미늄겔정          9          9      0.797          1      0.875      0.875
타이레놀이알서방정(아세트아미노펜)(수출

하이루

In [27]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt

# 예측 결과 이미지가 저장된 디렉토리
predict_dir = "runs/detect/predict"

# 예측된 모든 이미지 파일 불러오기
all_images = [f for f in os.listdir(predict_dir) if f.lower().endswith((".jpg", ".png"))]

# 20개 이미지 무작위 선택
sample_images = random.sample(all_images, min(20, len(all_images)))

# 시각화 (10행 2열 = 한 줄에 2개씩 20개)
plt.figure(figsize=(10, 40))  # 너비=10, 높이=40으로 크게 설정

for i, img_file in enumerate(sample_images):
    img_path = os.path.join(predict_dir, img_file)
    img = Image.open(img_path)

    plt.subplot(10, 2, i + 1)
    plt.imshow(img)
    plt.title(img_file)
    plt.axis("off")

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [28]:
print(len(train_results))  # 예측된 이미지 수
print(train_results[0].boxes)  # 첫 번째 이미지의 바운딩 박스 정보

1192
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([23., 59.,  0.], device='cuda:0')
conf: tensor([0.9559, 0.8276, 0.7956], device='cuda:0')
data: tensor([[5.6936e+02, 1.2457e+02, 8.4997e+02, 5.8350e+02, 9.5594e-01, 2.3000e+01],
        [1.3552e+02, 1.2700e+02, 3.7965e+02, 5.4684e+02, 8.2758e-01, 5.9000e+01],
        [6.4036e+02, 8.6191e+02, 8.5509e+02, 1.0169e+03, 7.9559e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1280, 976)
shape: torch.Size([3, 6])
xywh: tensor([[709.6660, 354.0344, 280.6042, 458.9298],
        [257.5858, 336.9183, 244.1323, 419.8446],
        [747.7205, 939.4113, 214.7300, 154.9973]], device='cuda:0')
xywhn: tensor([[0.7271, 0.2766, 0.2875, 0.3585],
        [0.2639, 0.2632, 0.2501, 0.3280],
        [0.7661, 0.7339, 0.2200, 0.1211]], device='cuda:0')
xyxy: tensor([[ 569.3638,  124.5695,  849.9681,  583.4993],
        [ 135.5196,  126.9960,  379.6519,  546.8406],
        [ 640.3555,  861.9127,  855.0854, 1016.9100

In [35]:
import os
import shutil
from glob import glob

# 원본 테스트 이미지 디렉토리
# test_image_dir = './ai02_data/test_images'

# YOLO 포맷 저장 경로
# output_base = './yolo_dataset'
img_test_dir = os.path.join(output_base, 'images', 'test')

# 기존 test 폴더 삭제
if os.path.exists(img_test_dir):
    shutil.rmtree(img_test_dir)
    print(f"기존 디렉토리 삭제됨: {img_test_dir}")

# 디렉토리 새로 생성
os.makedirs(img_test_dir, exist_ok=True)

# 이미지 경로 숫자 기준 정렬
test_image_paths = sorted(
    glob(os.path.join(test_image_dir, '*.png')),
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
)

# 복사
for img_path in test_image_paths:
    file_name = os.path.basename(img_path)
    shutil.copy2(img_path, os.path.join(img_test_dir, file_name))

print(f"Test 이미지 {len(test_image_paths)}개가 '{img_test_dir}'에 복사되었습니다.")

기존 디렉토리 삭제됨: ./yolo_dataset/images/test
Test 이미지 843개가 './yolo_dataset/images/test'에 복사되었습니다.


In [37]:
from google.colab import files
files.upload()

{}

In [38]:
test_result = model.predict(
    source="/content/drive/MyDrive/codeit_level1_project/data/ai02-level1-project/test_images",  # test 경로로 수정
    conf=0.05,
    #imgsz=640,
    save=False,
    save_txt=False,
    name="predict_test"
)


image 1/843 /content/drive/MyDrive/codeit_level1_project/data/ai02-level1-project/test_images/1.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 놀텍정 10mg, 1 울트라셋이알서방정, 9.5ms
image 2/843 /content/drive/MyDrive/codeit_level1_project/data/ai02-level1-project/test_images/10.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 라비에트정 20mg, 1 레일라정, 8.1ms
image 3/843 /content/drive/MyDrive/codeit_level1_project/data/ai02-level1-project/test_images/100.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 란스톤엘에프디티정 30mg, 1 신바로정, 9.6ms
image 4/843 /content/drive/MyDrive/codeit_level1_project/data/ai02-level1-project/test_images/1003.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 8.5ms
image 5/843 /content/drive/MyDrive/codeit_level1_project/data/ai02-level1-project/test_images/1004.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 8.1ms
image 6/843 /content/drive/MyDrive/codeit_level1_project/data/ai02-level1-project/test_images/1005.png: 640x5

In [32]:
from google.colab import files
files.upload()

Saving classid_to_categoryid.json to classid_to_categoryid.json


{'classid_to_categoryid.json': b'{\r\n  "0": 1899,\r\n  "1": 2482,\r\n  "2": 3350,\r\n  "3": 3482,\r\n  "4": 3543,\r\n  "5": 3742,\r\n  "6": 3831,\r\n  "7": 4377,\r\n  "8": 4542,\r\n  "9": 5093,\r\n  "10": 5885,\r\n  "11": 6191,\r\n  "12": 6562,\r\n  "13": 10220,\r\n  "14": 12080,\r\n  "15": 12246,\r\n  "16": 12419,\r\n  "17": 12777,\r\n  "18": 13394,\r\n  "19": 13899,\r\n  "20": 16231,\r\n  "21": 16261,\r\n  "22": 16547,\r\n  "23": 16550,\r\n  "24": 16687,\r\n  "25": 18109,\r\n  "26": 18146,\r\n  "27": 18356,\r\n  "28": 19231,\r\n  "29": 19551,\r\n  "30": 19606,\r\n  "31": 19860,\r\n  "32": 20013,\r\n  "33": 20237,\r\n  "34": 20876,\r\n  "35": 21025,\r\n  "36": 21324,\r\n  "37": 21770,\r\n  "38": 22073,\r\n  "39": 22346,\r\n  "40": 22361,\r\n  "41": 22626,\r\n  "42": 23202,\r\n  "43": 23222,\r\n  "44": 24849,\r\n  "45": 25366,\r\n  "46": 25437,\r\n  "47": 25468,\r\n  "48": 27652,\r\n  "49": 27732,\r\n  "50": 27776,\r\n  "51": 27925,\r\n  "52": 27992,\r\n  "53": 28762,\r\n  "54": 29344

In [39]:
from pathlib import Path
import pandas as pd

rows = []
annotation_id = 1

for result in test_result:
    # 올바른 image_id 추출 (파일명에서 숫자 부분만 가져오기)
    image_id = int(Path(result.path).stem)  # 예: '4.png' → 4

    boxes = result.boxes
    if boxes is None or boxes.xywh is None:
        continue

    for i in range(len(boxes.xywh)):
        x, y, w, h = boxes.xywh[i].tolist()
        score = float(boxes.conf[i])
        category_id = int(boxes.cls[i])  # 나중에 json 매핑 필요!

        xmin = round(x - w / 2)
        ymin = round(y - h / 2)
        bbox_w = round(w)
        bbox_h = round(h)

        rows.append([
            annotation_id,
            image_id,
            category_id,
            xmin,
            ymin,
            bbox_w,
            bbox_h,
            round(score, 2)
        ])
        annotation_id += 1

df = pd.DataFrame(rows, columns=[
    "annotation_id", "image_id", "category_id",
    "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"
])


# 카테고리 매핑 적용
with open("classid_to_categoryid.json", "r") as f:
    class_id_to_category_id = json.load(f)

class_id_to_category_id = {int(k): v for k, v in class_id_to_category_id.items()}
df["category_id"] = df["category_id"].map(class_id_to_category_id)

# 저장
csv_path = "/content/submission_mapped.csv"
df.to_csv(csv_path, index=False)
print(f"CSV 저장 완료: {csv_path} / 총 {len(df)}개의 바운딩 박스")

CSV 저장 완료: /content/submission_mapped.csv / 총 3259개의 바운딩 박스


In [40]:
from google.colab import files
files.download(csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import json

# 1. 기존 제출용 CSV 불러오기
df_submit = pd.read_csv("your_submission.csv")

# 2. JSON 매핑 파일 불러오기
with open("class_id_to_category_id.json", "r") as f:
    class_id_to_category_id = json.load(f)

# 3. 문자열 키를 정수로 바꿔줌
class_id_to_category_id = {int(k): v for k, v in class_id_to_category_id.items()}

# 4. 매핑 적용
df_submit["category_id"] = df_submit["category_id"].map(class_id_to_category_id)

# 5. 확인 후 저장
print(df_submit.head())
df_submit.to_csv("submission_mapped.csv", index=False)
